## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Khani,GE,41.9563,42.9566,63.97,98,100,2.46,light rain
1,1,Atar,MR,20.5169,-13.0499,99.73,24,0,13.65,clear sky
2,2,Yerbogachen,RU,61.2767,108.0108,51.28,94,36,4.38,scattered clouds
3,3,Kamaishi,JP,39.2667,141.8833,69.40,96,100,10.92,overcast clouds
4,4,Punta Arenas,CL,-53.1500,-70.9167,33.91,93,100,9.22,light snow


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("what is the minimum temperature you would like for your trip? "))
max_temp = float(input("what is the maximum temperature you would like for your trip? "))

what is the minimum temperature you would like for your trip? 65
what is the maximum temperature you would like for your trip? 75


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
select_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
select_cities_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Kamaishi,JP,39.2667,141.8833,69.40,96,100,10.92,overcast clouds
17,17,Mahebourg,MU,-20.4081,57.7000,68.29,83,75,12.66,broken clouds
18,18,Ribeira Grande,PT,38.5167,-28.7000,70.20,68,20,13.80,few clouds
21,21,Tebingtinggi,ID,3.3285,99.1625,74.84,90,100,2.89,overcast clouds
28,28,Vaini,TO,-21.2000,-175.2000,71.76,83,20,8.05,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
select_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136 entries, 3 to 663
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           136 non-null    int64  
 1   City                 136 non-null    object 
 2   Country              134 non-null    object 
 3   Lat                  136 non-null    float64
 4   Lng                  136 non-null    float64
 5   Max Temp             136 non-null    float64
 6   Humidity             136 non-null    int64  
 7   Cloudiness           136 non-null    int64  
 8   Wind Speed           136 non-null    float64
 9   Current Description  136 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 11.7+ KB


In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = select_cities_df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134 entries, 3 to 663
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           134 non-null    int64  
 1   City                 134 non-null    object 
 2   Country              134 non-null    object 
 3   Lat                  134 non-null    float64
 4   Lng                  134 non-null    float64
 5   Max Temp             134 non-null    float64
 6   Humidity             134 non-null    int64  
 7   Cloudiness           134 non-null    int64  
 8   Wind Speed           134 non-null    float64
 9   Current Description  134 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 11.5+ KB


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kamaishi,JP,69.40,overcast clouds,39.2667,141.8833,
17,Mahebourg,MU,68.29,broken clouds,-20.4081,57.7000,
18,Ribeira Grande,PT,70.20,few clouds,38.5167,-28.7000,
21,Tebingtinggi,ID,74.84,overcast clouds,3.3285,99.1625,
28,Vaini,TO,71.76,few clouds,-21.2000,-175.2000,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params ["location"] = F"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123 entries, 3 to 663
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 123 non-null    object 
 1   Country              123 non-null    object 
 2   Max Temp             123 non-null    float64
 3   Current Description  123 non-null    object 
 4   Lat                  123 non-null    float64
 5   Lng                  123 non-null    float64
 6   Hotel Name           123 non-null    object 
dtypes: float64(3), object(4)
memory usage: 7.7+ KB


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
""" 

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))